In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
import os
import gc
from keras.preprocessing.image import image,load_img,img_to_array
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
#processing the image and converting them into numbers
#getting all images
#train_pics = image.list_pictures("train/")
test_pics = image.list_pictures("test/")

def image_resize(list,folder):
    #convert all to the same dimensions of 100*100
    os.mkdir(folder)
    i = 0
    for im in list:
        #get old name
        name = im.split("/")[1]
        try:
            #read the image
            z = load_img(im)
            #resize the image
            z = z.resize([64,64])
            #save the image
            z.save(folder+name)
            i += 1
            if (i %500 == 0 ):
                print("{} images converted".format(i))
        except:
            print("Cannot process {}".format(name))
    print("Conversion Completed")
#print("training set")
#image_resize(train_pics,"cotrain/")
print("testing set")
image_resize(test_pics,"cotest/")

testing set
Cannot process .DS.png
500 images converted
1000 images converted
Conversion Completed


In [6]:
#train = image.list_pictures("cotrain/")
test = image.list_pictures("cotest/")
#instatiate the test and train images
#d1 = np.zeros([len(train),64,64,3])
d2 = np.zeros([len(test),64,64,3])
def convert_to_numpy(file,list):
    i = 0
    for im in file:
        #read the image
        try:
            z = load_img(im)
            #convert into images and replace in the instatiated datasets above
            list[i] = img_to_array(z)
            i += 1
            if (i%500==0):
                print("{} images converted".format(i))
        except:
            print("Cannot process image {}".format(im))
    print ("Conversion is Completed")
#print("Training Set")
#convert_to_numpy(train,d1)
print("Testing Set")
convert_to_numpy(test,d2)

Testing Set
500 images converted
1000 images converted
Conversion is Completed


In [7]:
# #normalizing the data
d1 = (d1 -128)/ 128
d2 = (d2 - 128)/ 128

In [8]:
#getting the id for images accordingly
img_id = []
img_id2 = []
for dir in train:
    img_id.append(dir.split("/")[1].split(".")[0])
img_id_df = pd.Series(img_id,name='img_id').astype("int64")

for dir in test:
    img_id.append(dir.split("/")[1].split(".")[0])
img_id_df2 = pd.Series(img_id2,name='img_id').astype("int64")

#reshape your image and join
dt1 = d1.reshape([d1.shape[0],64*64*3])
dt2 = d2.reshape([d2.shape[0],64*64*3])
dt_df = pd.DataFrame(dt1)
dt_df2 = pd.DataFrame(dt2)

del dt1
del dt2
del img_id
del img_id2

#joining the dataframe together 
dt = dt_df.join(img_id_df)
dt2 = dt_df2.join(img_id_df2)

del dt_df
del dt_df2
del img_id_df
del img_id_df2
gc.collect()

42

,0,1,2,3,4,5,6,7,8,9,...,12279,12280,12281,12282,12283,12284,12285,12286,12287,img_id
0,0.687500,0.632812,0.539062,0.695312,0.640625,0.554688,0.710938,0.632812,0.554688,0.718750,...,0.578125,0.500000,0.429688,0.546875,0.468750,0.390625,0.523438,0.437500,0.390625,NaN
1,-0.289062,-0.312500,-0.367188,0.609375,0.687500,0.820312,0.617188,0.703125,0.781250,0.609375,...,-0.421875,-0.453125,-0.546875,-0.421875,-0.437500,-0.554688,-0.429688,-0.460938,-0.546875,NaN
2,0.398438,0.257812,0.085938,0.335938,0.195312,0.023438,0.218750,0.078125,-0.109375,0.328125,...,0.375000,0.234375,0.031250,0.343750,0.203125,0.015625,0.351562,0.210938,0.023438,NaN
3,0.546875,0.421875,0.226562,0.531250,0.406250,0.210938,0.554688,0.429688,0.234375,0.515625,...,0.421875,0.289062,0.140625,0.437500,0.304688,0.156250,0.453125,0.320312,0.179688,NaN
4,0.406250,0.312500,0.289062,0.531250,0.429688,0.421875,0.539062,0.445312,0.445312,0.195312,...,-0.750000,-0.750000,-0.734375,-0.757812,-0.757812,-0.742188,-0.734375,-0.734375,-0.718750,NaN


In [16]:
#reading the labels datasets
id_df = pd.read_csv("train_labels.csv")
id_df.drop('Unnamed: 0',axis=1,inplace=True)

#merge the two datasets for proper labeling
final_train_df = dt.merge(id_df,on="img_id")

#---------------------------------------------------------------------#

#reading the labels datasets
id_df2 = pd.read_csv("test_ids.csv")

#merge the two datasets for proper labeling
final_test_df = dt2.merge(id_df2,on="img_id")

In [ ]:
#final_train_df = pd.read_csv("training.csv")

X = final_train_df.drop(['fuel','parking','restaurant','transport','img_id'],axis=1)
y = final_train_df[['fuel','parking','restaurant','transport']]

In [ ]:
#train test split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=7)

In [ ]:
#converting back to numpy and reshaping
Xtrain = X_train.values.reshape([len(X_train),64,64,3])
Xtest = X_test.values.reshape([len(X_test),64,64,3])
ytrain = y_train.values
ytest = y_test.values

In [52]:
# #an MlP model

# model = Sequential()

# model.add(Dense(256,activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(128,activation="relu"))
# model.add(Dropout(0.25))
# #model.add(Dense(32,activation="relu"))
# #model.add(Dropout(0.5))
# model.add(Dense(4,activation="softmax"))

In [ ]:
#structure of your convolution layer

model = Sequential()
model.add(Conv2D(32,(3,3),activation="relu",input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(32,(5,5),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dropout(rate=0.75))
model.add(Dense(1024,activation="relu"))
model.add(Dropout(0.75))
model.add(Dense(4,activation="softmax"))

In [ ]:
model.compile(optimizer="adam",loss="mse",metrics=['accuracy'])

In [ ]:
model.fit(Xtrain,ytrain,batch_size=128,epochs=15)

In [51]:
#model evaluating is below this

In [52]:
pred = model.predict(Xtest)

In [53]:
a = accuracy_score(ytest.argmax(1),pred.argmax(1))

In [54]:
a

0.568407960199005

In [195]:
#0.5 model approximately

"""

using a rescale of 32*32

#structure of your convolution layer

model = Sequential()
model.add(Conv2D(16,(3,3),activation="relu",input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(32,(5,5),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dropout(rate=0.25))
model.add(Dense(4,activation="softmax"))

a catcrossent as loss function

and an adam opt..

""""
"""
this performed better on the test set than on the training sets
using dropouts of 75% on the mlp

#structure of your convolution layer

model = Sequential()
model.add(Conv2D(16,(3,3),activation="relu",input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(16,(5,5),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dropout(rate=0.75))
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.75))
model.add(Dense(4,activation="softmax"))


"""

"""

55% on the testing set

#structure of your convolution layer

model = Sequential()
model.add(Conv2D(32,(3,3),activation="relu",input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(32,(5,5),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dropout(rate=0.75))
model.add(Dense(1024,activation="relu"))
model.add(Dropout(0.75))
model.add(Dense(4,activation="softmax"))

"""

#things to note based on the tuning

#1)More nodes in a layer is better than a deeper layer

#2)adam is the best optimizer

#3)with probably categorical_crossentropy as the loss function

#4)